In [1]:
%matplotlib widget
from util import get_path, get_dates_datetime
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from extract_graph import generate_nx_graph, transform_list, generate_skeleton, generate_nx_graph_from_skeleton, from_connection_tab, from_nx_to_tab
from node_id import whole_movement_identification, second_identification
import ast
from plotutil import plot_t_tp1, compress_skeleton
from scipy import sparse
from sparse_util import dilate, zhangSuen
from realign import realign
from datetime import datetime,timedelta
from node_id import orient
import pickle
from matplotlib.widgets import CheckButtons
import scipy.io as sio
import imageio
from pymatreader import read_mat
from matplotlib import colors
from copy import deepcopy,copy
from collections import Counter
import cv2 
import imageio
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import frangi, meijering
from skimage.morphology import thin
from skimage import data, filters
from random import choice
import scipy.sparse
import os
from time import time
from extract_graph import dic_to_sparse, from_sparse_to_graph, generate_nx_graph, prune_graph, from_nx_to_tab, from_nx_to_tab_matlab,sparse_to_doc, connections_pixel_list_to_tab, transform_list, clean_degree_4
from time import sleep
from skimage.feature import hessian_matrix_det
from experiment_class_surf import Experiment,clean_exp_with_hyphaes
from hyphae_id_surf import clean_and_relabel, get_mother, save_hyphaes, resolve_ambiguity_two_ends,solve_degree4, clean_obvious_fake_tips


***Chose plate number***

In [2]:
plate = 3

***Chose directory of interest where the folders with images are***

- the output of this  cell is the number of folders of interest where images can be found

In [3]:
directory = "/scratch/shared/mrozemul/Fiji.app/" 
listdir=os.listdir(directory) 
list_dir_interest=[name for name in listdir if name.split('_')[-1]==f'Plate{0 if plate<10 else ""}{plate}']
len(list_dir_interest)

20

***Chose timestep to begin with and folder to end with***

In [4]:
dates_datetime = get_dates_datetime(directory,plate)
begin = 0
end = 19
print('begin =',dates_datetime[begin],'\n  end =',dates_datetime[end])

begin = 2020-08-14 00:57:00 
  end = 2020-08-17 04:57:00


***Compress raw image***

In [31]:
num_parallel = 4
begin_skel = begin//num_parallel
end_skel = (end+1)//num_parallel
for j in range(begin_skel,end_skel):
    ide=int(datetime.now().timestamp())
    start = num_parallel*j
    stop = num_parallel*j+num_parallel-1
    my_file = open("/scratch/shared/mrozemul/Fiji.app/extract_skel.sh", "w")
    my_file.write("#!/bin/bash \n#Set job requirements \n#SBATCH -N 1 \n#SBATCH -t 2:00\n#SBATCH -p normal\n")
    my_file.write(f'#SBATCH -o "/home/mrozemul/Corentin/MscThesis/slurm/compress_{plate}_{4*j}_{4*j+3}_{ide}.out" \n')
    my_file.write(f'for i in `seq {4*j} {4*j+3}`; do\n')
    my_file.write(f'\t python /home/mrozemul/Corentin/MscThesis/compress_image.py $i {plate} &\n')
    my_file.write('done\n')
    my_file.write('wait\n')
    my_file.close()
    !sbatch /scratch/shared/mrozemul/Fiji.app/extract_skel.sh

Submitted batch job 9449576
Submitted batch job 9449577
Submitted batch job 9449578
Submitted batch job 9449579
Submitted batch job 9449580


***Skeletonization***
- Only 4 skeletonization processes can be run in parallel on one node

In [13]:
num_parallel = 4
begin_skel = begin//num_parallel
end_skel = (end+1)//num_parallel
for j in range(begin_skel,end_skel):
    ide=int(datetime.now().timestamp())
    start = num_parallel*j
    stop = num_parallel*j+num_parallel-1
    my_file = open(f'{directory}extract_skel.sh', "w")
    my_file.write("#!/bin/bash \n#Set job requirements \n#SBATCH -N 1 \n#SBATCH -t 2:00:00\n#SBATCH -p normal\n")
    my_file.write(f'#SBATCH -o "/home/mrozemul/Corentin/MscThesis/slurm/skeletonization_{plate}_{start}_{stop}_{ide}.out" \n')
    my_file.write(f'for i in `seq {start} {stop}`; do\n')
    my_file.write(f'\t python ~/Corentin/MscThesis/extract_skel_surf.py $i {plate} &\n')
    my_file.write('done\n')
    my_file.write('wait\n')
    my_file.close()
# !srun -t 2:00:00 --pty /scratch/shared/mrozemul/Fiji.app/extract_skel.sh
    !sbatch /scratch/shared/mrozemul/Fiji.app/extract_skel.sh

Submitted batch job 9449524
Submitted batch job 9449525
Submitted batch job 9449526
Submitted batch job 9449527
Submitted batch job 9449528


***Check Skeletonization***
- The first cell loads the skeleton and the compressed raw image
- The second cell shows the overlap of the skeleton and the raw image

In [46]:
dates_datetime = get_dates_datetime(directory,plate)
dates_datetime_chosen=dates_datetime[begin:end+1]
dates = [f'{0 if date.month<10 else ""}{date.month}{0 if date.day<10 else ""}{date.day}_{0 if date.hour<10 else ""}{date.hour}{0 if date.minute<10 else ""}{date.minute}' for date in dates_datetime_chosen]
skels = []
ims = []
kernel = np.ones((5,5),np.uint8)
itera = 1
for date in dates:
    directory_name=f'2020{date}_Plate{0 if plate<10 else ""}{plate}'
    path_snap=directory+directory_name
    skel_info = read_mat(path_snap+'/Analysis/skeleton_compressed.mat')
    skel = skel_info['skeleton']
    skels.append(cv2.dilate(skel.astype(np.uint8),kernel,iterations = itera))
    im = read_mat(path_snap+'/Analysis/raw_image.mat')['raw']
    ims.append(im)

- Chose start and finish to display the overlap of the skeleton and the raw image : no more than 10 at a time or jupyter will crash.
- For display purpose, everything is compressed so connectivity may be lost
- colors are just a feature of compression

In [47]:
plt.close('all')
start = 0
finish = start +5
for i in range(start,finish):
    plot_t_tp1([],[],None,None,skels[i],ims[i])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

***Check specific image***
- If something wrong is noticed in one of the skeletons one can chose to look closer at one of the images
- chose ***i*** equal to the timestep where something wrong has been noticed 

In [5]:
#chose i equal to the timestep where something wrong has been noticed 
i = 0

dates_datetime = get_dates_datetime(directory,plate)
dates_datetime_chosen=dates_datetime
dates = [f'{0 if date.month<10 else ""}{date.month}{0 if date.day<10 else ""}{date.day}_{0 if date.hour<10 else ""}{date.hour}{0 if date.minute<10 else ""}{date.minute}' for date in dates_datetime_chosen]
date =dates [i]
directory_name=f'2020{date}_Plate{0 if plate<10 else ""}{plate}'
path_snap=directory+directory_name
path_tile=path_snap+'/Img/TileConfiguration.txt.registered'
try:
    tileconfig = pd.read_table(path_tile,sep=';',skiprows=4,header=None,converters={2 : ast.literal_eval},skipinitialspace=True)
except:
    print('error_name')
    path_tile=path_snap+'/Img/TileConfiguration.registered.txt'
    tileconfig = pd.read_table(path_tile,sep=';',skiprows=4,header=None,converters={2 : ast.literal_eval},skipinitialspace=True)
t=time()
xs =[c[0] for c in tileconfig[2]]
ys =[c[1] for c in tileconfig[2]]
dim = (int(np.max(ys)-np.min(ys))+4096,int(np.max(xs)-np.min(xs))+4096)
ims = []
for name in tileconfig[0]:
#     ims.append(imageio.imread('//sun.amolf.nl/shimizu-data/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE'+date_plate+plate_str+'/Img/'+name))
    ims.append(imageio.imread(f'{name}'))


- Chose a x,y position where you want to see how the skeletonization process went (x is the scale on the left on the images and y is the bottom scale)
- You can chose to display different part of the filter par commenting/uncommenting

In [6]:
plt.close('all')

#chose a spot where to look closer at
linex  = 3500
liney = 1500

shape = (3000,4096)
linex *= 5
liney *= 5
for index,im in enumerate(ims):
    boundaries = int(tileconfig[2][index][0]-np.min(xs)),int(tileconfig[2][index][1]-np.min(ys))
    if boundaries[1]<=linex< boundaries[1]+shape[0] and boundaries[0]<=liney< boundaries[0]+shape[1]:
        print(index)
        im_cropped = im
        im_blurred =cv2.blur(im_cropped, (200, 200))
        im_back_rem = (im_cropped+1)/(im_blurred+1)*120
        # # im_back_rem = im_cropped*1.0
        # # # im_back_rem = cv2.normalize(im_back_rem, None, 0, 255, cv2.NORM_MINMAX)
        frangised = frangi(im_back_rem,sigmas=range(1,20,4))*255
        # # frangised = cv2.normalize(frangised, None, 0, 255, cv2.NORM_MINMAX)
        hessian = hessian_matrix_det(im_back_rem,sigma = 20)
        blur_hessian = cv2.blur(abs(hessian), (20, 20))
        #     transformed = (frangised+cv2.normalize(blur_hessian, None, 0, 255, cv2.NORM_MINMAX)-im_back_rem+120)*(im_blurred>=35)
        #     transformed = (frangised+cv2.normalize(abs(hessian), None, 0, 255, cv2.NORM_MINMAX)-im_back_rem+120)*(im_blurred>=35)
        transformed = (frangised-im_back_rem+120)*(im_blurred>=35)
        low = 20
        high = 100
        lowt = (transformed > low).astype(int)
        hight = (transformed > high).astype(int)
        hyst = filters.apply_hysteresis_threshold(transformed, low, high)
        kernel = np.ones((3,3),np.uint8)
        dilation = cv2.dilate(hyst.astype(np.uint8) * 255,kernel,iterations = 1)
        for i in range(3):
            dilation=cv2.erode(dilation.astype(np.uint8) * 255,kernel,iterations = 1)
            dilation = cv2.dilate(dilation.astype(np.uint8) * 255,kernel,iterations = 1)
        dilated = dilation>0
        skeletonized = cv2.ximgproc.thinning(np.array(255*dilated,dtype=np.uint8))
#         hessian = hessian_matrix_det(dilated,sigma = 15)
#         hessian = cv2.normalize(-hessian, None, 0, 255, cv2.NORM_MINMAX)
#         kernel = np.ones((5,5),np.uint8)
#         junction_connecter = hessian>165
#         junction_connecter =  cv2.dilate(junction_connecter.astype(np.uint8) * 255,kernel,iterations = 5)

        print('Raw image')
#         fig=plt.figure(figsize=(10,9))
#         ax = fig.add_subplot(111)
#         ax.imshow(im)
        print('Raw image with background removed')
        fig=plt.figure(figsize=(10,9))
        ax = fig.add_subplot(111)
        ax.imshow(im_back_rem)
#         print('frangised image')
#         fig=plt.figure(figsize=(10,9))
#         ax = fig.add_subplot(111)
#         ax.imshow(frangised)
#         print('final transformed image')
        fig=plt.figure(figsize=(10,9))
        ax = fig.add_subplot(111)
        ax.imshow(transformed)
#         print('threhsolded image')
#         fig=plt.figure(figsize=(10,9))
#         ax = fig.add_subplot(111)
#         ax.imshow(dilated-junction_connecter>0)
        fig=plt.figure(figsize=(10,9))
        ax = fig.add_subplot(111)
        ax.imshow(dilated)
#         fig=plt.figure(figsize=(10,9))
#         ax = fig.add_subplot(111)
#         ax.imshow(cv2.normalize(abs(hessian), None, 0, 255, cv2.NORM_MINMAX)-255*dilated)
        plot_t_tp1([],[],None,None,dilated,skeletonized)

79
Raw image
Raw image with background removed


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [41]:
plt.close('all')
transformed = (frangised-im_back_rem+120)*(im_blurred>=35)
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
test=np.copy(im_back_rem)
test[test >= 130] =130
# im_back_rem = (im_cropped+1)/(im_blurred+1)*120
ax.imshow(test)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
hessian = hessian_matrix_det(frangised,sigma = 15)
hessian = cv2.normalize(-hessian, None, 0, 255, cv2.NORM_MINMAX)

In [28]:
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.imshow(transformed)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [57]:
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.imshow(hessian)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

***Mask baits and border of the petri dish***

In [14]:
for j in range(0,6):
    ide=int(datetime.now().timestamp())
    my_file = open("/scratch/shared/mrozemul/Fiji.app/extract_skel.sh", "w")
    my_file.write("#!/bin/bash \n#Set job requirements \n#SBATCH -N 1 \n#SBATCH -t 5:00\n#SBATCH -p normal\n")
    my_file.write(f'#SBATCH -o "/home/mrozemul/Corentin/MscThesis/slurm/mask_{plate}_{4*j}_{4*j+3}_{ide}.out" \n')
    my_file.write(f'for i in `seq {4*j} {4*j+3}`; do\n')
    my_file.write(f'\t python /home/mrozemul/Corentin/MscThesis/mask_skel.py $i {plate} &\n')
    my_file.write('done\n')
    my_file.write('wait\n')
    my_file.close()
    !sbatch /scratch/shared/mrozemul/Fiji.app/extract_skel.sh

Submitted batch job 9448827
Submitted batch job 9448828
Submitted batch job 9448829
Submitted batch job 9448830
Submitted batch job 9448831
Submitted batch job 9448832


***Realign***

In [4]:
for j in range(0,8):
    ide=int(datetime.now().timestamp())
    my_file = open("/scratch/shared/mrozemul/Fiji.app/align.sh", "w")
    my_file.write("#!/bin/bash \n#Set job requirements \n#SBATCH -N 1 \n#SBATCH -t 2:00:00\n#SBATCH -p normal\n")
    my_file.write(f'for i in `seq {8*j} {8*j+7}`; do\n')
    my_file.write(f'\t python ~/Corentin/MscThesis/final_alignment.py $i {plate} &\n')
    my_file.write('done\n')
    my_file.write('wait\n')
    my_file.close()
# !srun -t 2:00:00 --pty /scratch/shared/mrozemul/Fiji.app/align.sh
    !sbatch /scratch/shared/mrozemul/Fiji.app/align.sh

Submitted batch job 9446451
Submitted batch job 9446452
Submitted batch job 9446453
Submitted batch job 9446454
Submitted batch job 9446455
Submitted batch job 9446456
Submitted batch job 9446457
Submitted batch job 9446458


***Check Alignment***

In [87]:
plt.close('all')
directory = "/scratch/shared/mrozemul/Fiji.app/" 
listdir=os.listdir(directory) 
list_dir_interest=[name for name in listdir if name.split('_')[-1]==f'Plate{0 if plate<10 else ""}{plate}']
ss=[name.split('_')[0] for name in list_dir_interest]
ff=[name.split('_')[1] for name in list_dir_interest]
dates_datetime=[datetime(year=int(ss[i][:4]),month=int(ss[i][4:6]),day=int(ss[i][6:8]),hour=int(ff[i][0:2]),minute=int(ff[i][2:4])) for i in range(len(list_dir_interest))]
dates_datetime.sort()
dates_datetime_chosen=dates_datetime[begin:end+1]
dates = [f'{0 if date.month<10 else ""}{date.month}{0 if date.day<10 else ""}{date.day}_{0 if date.hour<10 else ""}{date.hour}{0 if date.minute<10 else ""}{date.minute}' for date in dates_datetime_chosen]
dilateds=[]
skels = []
skel_docs = []
directory_name=f'2020{dates[0]}_Plate{0 if plate<10 else ""}{plate}'
path_snap='/scratch/shared/mrozemul/Fiji.app/'+directory_name
skel_info = read_mat(path_snap+'/Analysis/skeleton.mat')
skel = skel_info['skeleton']
skels.append(skel)
skel_doc = sparse_to_doc(skel)
skel_docs.append(skel_doc)
Rs=[]
ts=[]
for date in dates[1:]:
    directory_name=f'2020{date}_Plate{0 if plate<10 else ""}{plate}'
    path_snap='/scratch/shared/mrozemul/Fiji.app/'+directory_name
    skel_info = read_mat(path_snap+'/Analysis/skeleton.mat')
    skel = skel_info['skeleton']
    skels.append(skel)
    skel_doc = sparse_to_doc(skel)
    skel_docs.append(skel_doc)
    transform = sio.loadmat(path_snap+'/Analysis/transform.mat')
    R,t = transform['R'],transform['t']
    Rs.append(R)
    ts.append(t)
# start = 0
# for j in range(start,start + 5):
#     print(dates[j],j+begin)
#     skeleton1,skeleton2 = skel_docs[j],skel_docs[j+1]
#     R,t = Rs[j],ts[j]
#     skelet_pos = np.array(list(skeleton1.keys()))
#     samples = np.random.choice(skelet_pos.shape[0],20000)
#     X = np.transpose(skelet_pos[samples,:])
#     skelet_pos = np.array(list(skeleton2.keys()))
#     samples = np.random.choice(skelet_pos.shape[0],20000)
#     Y = np.transpose(skelet_pos[samples,:])
#     fig=plt.figure(figsize=(10,9))
#     ax = fig.add_subplot(111)
#     Yrep=np.transpose(np.transpose(np.dot(R,X))+t)
#     ax.scatter(np.transpose(Yrep)[:,0],np.transpose(Yrep)[:,1])
#     ax.scatter(np.transpose(Y)[:,0],np.transpose(Y)[:,1])

***Create realigned Skeleton***

In [57]:
num_parallel = 5
begin_skel = begin//num_parallel
end_skel = (end+1)//num_parallel
for j in range(begin_skel,end_skel):
    start = num_parallel*j
    stop = num_parallel*j+num_parallel-1
    my_file = open("/scratch/shared/mrozemul/Fiji.app/align_skelt.sh", "w")
    my_file.write("#!/bin/bash \n#Set job requirements \n#SBATCH -N 1 \n#SBATCH -t 1:00:00\n#SBATCH -p normal\n")
    my_file.write(f'#SBATCH -o "/home/mrozemul/Corentin/MscThesis/slurm/create_realign_{plate}_{start}_{stop}_{ide}.out" \n')
    my_file.write(f'for i in `seq {start} {stop}`; do\n')
    my_file.write(f'python ~/Corentin/MscThesis/realign_surf.py {plate} {begin} {end} $i &\n')
    my_file.write('done\n')
    my_file.write('wait\n')
    my_file.close()
# !srun -t 2:00:00 --pty /scratch/shared/mrozemul/Fiji.app/align_skelt.sh
    !sbatch /scratch/shared/mrozemul/Fiji.app/align_skelt.sh

Submitted batch job 9449002
Submitted batch job 9449003
Submitted batch job 9449004
Submitted batch job 9449005


***Check Fine Alignment***

In [5]:
plt.close('all')
directory = "/scratch/shared/mrozemul/Fiji.app/" 
listdir=os.listdir(directory) 
list_dir_interest=[name for name in listdir if name.split('_')[-1]==f'Plate{0 if plate<10 else ""}{plate}']
ss=[name.split('_')[0] for name in list_dir_interest]
ff=[name.split('_')[1] for name in list_dir_interest]
dates_datetime=[datetime(year=int(ss[i][:4]),month=int(ss[i][4:6]),day=int(ss[i][6:8]),hour=int(ff[i][0:2]),minute=int(ff[i][2:4])) for i in range(len(list_dir_interest))]
dates_datetime.sort()
begin = 0
end = 20
dates_datetime_chosen=dates_datetime[begin:end]
dates = [f'{0 if date.month<10 else ""}{date.month}{0 if date.day<10 else ""}{date.day}_{0 if date.hour<10 else ""}{date.hour}{0 if date.minute<10 else ""}{date.minute}' for date in dates_datetime_chosen]
zone = (13000,13000+5000+3000,20000,20000+5000+4096)
skels_aligned = []
for i,date in enumerate(dates):
    directory_name=f'2020{dates[i]}_Plate{0 if plate<10 else ""}{plate}'
    path_snap='/scratch/shared/mrozemul/Fiji.app/'+directory_name
    skels_aligned.append(sio.loadmat(path_snap+'/Analysis/skeleton_realigned.mat')['skeleton'])
for i in range(11,13):
    plot_t_tp1([],[],None,None,skels_aligned[i][zone[0]:zone[1],zone[2]:zone[3]].todense(),skels_aligned[i+1][zone[0]:zone[1],zone[2]:zone[3]].todense())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
plt.close('all')
zone = (6000,13000,12000,22000)
fig=plt.figure(figsize=(10,9))
ax = fig.add_subplot(111)
ax.imshow(skels_aligned[11][zone[0]:zone[1],zone[2]:zone[3]].todense())

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

***Prune Graph***

In [58]:
threshold = 1
num_parallel = 20
begin_skel = begin//num_parallel
end_skel = (end+1)//num_parallel
for j in range(begin_skel,end_skel):
    ide=int(datetime.now().timestamp())
    start = num_parallel*j
    stop = num_parallel*j+num_parallel-1
    my_file = open("/scratch/shared/mrozemul/Fiji.app/nx_graph_extr.sh", "w")
    my_file.write("#!/bin/bash \n#Set job requirements \n#SBATCH -N 1 \n#SBATCH -t 20:00\n#SBATCH -p normal\n")
    my_file.write(f'#SBATCH -o "/home/mrozemul/Corentin/MscThesis/slurm/prune_graph_{plate}_{start}_{stop}_{ide}.out" \n')
    my_file.write(f'for i in `seq {start} {stop}`; do\n')
    my_file.write(f'\t python ~/Corentin/MscThesis/extract_nx_graph.py {plate} $i {threshold}&\n')
    my_file.write('done\n')
    my_file.write('wait\n')
    my_file.close()
    !sbatch /scratch/shared/mrozemul/Fiji.app/nx_graph_extr.sh
# !srun -t 2:00:00 --pty /scratch/shared/mrozemul/Fiji.app/nx_graph_extr.sh


Submitted batch job 9449098


***Check Pruned Graphs***

In [64]:
start = 0
stop = 2
directory = "/scratch/shared/mrozemul/Fiji.app/" 
listdir=os.listdir(directory) 
list_dir_interest=[name for name in listdir if name.split('_')[-1]==f'Plate{0 if plate<10 else ""}{plate}']
ss=[name.split('_')[0] for name in list_dir_interest]
ff=[name.split('_')[1] for name in list_dir_interest]
dates_datetime=[datetime(year=int(ss[i][:4]),month=int(ss[i][4:6]),day=int(ss[i][6:8]),hour=int(ff[i][0:2]),minute=int(ff[i][2:4])) for i in range(len(list_dir_interest))]
dates_datetime.sort()
dates_datetime_chosen=dates_datetime[start:stop]
dates = [f'{0 if date.month<10 else ""}{date.month}{0 if date.day<10 else ""}{date.day}_{0 if date.hour<10 else ""}{date.hour}{0 if date.minute<10 else ""}{date.minute}' for date in dates_datetime_chosen]
nx_graph_pos = []
skeletons = []
for date in dates:
    directory_name=f'2020{date}_Plate{0 if plate<10 else ""}{plate}'
    path_snap='/scratch/shared/mrozemul/Fiji.app/'+directory_name
    path_save = path_snap +'/Analysis/nx_graph_pruned.p'
    nx_graph_pos.append(pickle.load(open( path_save, "rb" )))
    skeletons.append(scipy.sparse.dok_matrix(sio.loadmat(path_snap+'/Analysis/skeleton_realigned_pruned.mat')['skeleton']))
compressed = [compress_skeleton(skeleton,5) for skeleton in skeletons]

In [65]:
plt.close('all')
kernel = np.ones((5,5),np.uint8)
for i in range(len(compressed)-1):
    plot_t_tp1([],[],None,None,cv2.dilate(compressed[i].astype(np.uint8),kernel,iterations = 2),cv2.dilate(compressed[i+1].astype(np.uint8),kernel,iterations = 2))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

***Identify Nodes***

In [39]:
ide=int(datetime.now().timestamp())
my_file = open("/scratch/shared/mrozemul/Fiji.app/extract_nodes.sh", "w")
my_file.write("#!/bin/bash \n#Set job requirements \n#SBATCH -N 1 \n#SBATCH -t 24:00:00\n#SBATCH -p normal\n")
my_file.write(f'#SBATCH -o "/home/mrozemul/Corentin/MscThesis/slurm/node_id_{plate}_{begin}_{end}_{ide}.out" \n')
my_file.write(f'python ~/Corentin/MscThesis/extract_nodes_surf.py {plate} {begin} {end+1}&\n')
my_file.write('wait\n')
my_file.close()
!sbatch /scratch/shared/mrozemul/Fiji.app/extract_nodes.sh
# !srun -t 2:00:00 --pty /scratch/shared/mrozemul/Fiji.app/extract_nodes.sh

Submitted batch job 9449628


***Check Node Id***

In [9]:
plate = 3

In [28]:
begin = 0
end = 5
directory = "/scratch/shared/mrozemul/Fiji.app/" 
listdir=os.listdir(directory) 
list_dir_interest=[name for name in listdir if name.split('_')[-1]==f'Plate{0 if plate<10 else ""}{plate}']
ss=[name.split('_')[0] for name in list_dir_interest]
ff=[name.split('_')[1] for name in list_dir_interest]
dates_datetime=[datetime(year=int(ss[i][:4]),month=int(ss[i][4:6]),day=int(ss[i][6:8]),hour=int(ff[i][0:2]),minute=int(ff[i][2:4])) for i in range(len(list_dir_interest))]
dates_datetime.sort()
dates_datetime_chosen=dates_datetime[begin:end]
dates = [f'{0 if date.month<10 else ""}{date.month}{0 if date.day<10 else ""}{date.day}_{0 if date.hour<10 else ""}{date.hour}{0 if date.minute<10 else ""}{date.minute}' for date in dates_datetime_chosen]
exp= Experiment(plate)
exp.load(dates)

In [29]:
plt.close('all')
t = 2
nodes = np.random.choice(exp.nx_graph[t].nodes,100)
# exp.plot([t,t+1,t+2],[list(nodes)]*3)
exp.plot([t,t+1,t+2],[[5673]]*3)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

***Hyphae extraction***

In [33]:
plate = 3

In [34]:
ide=int(datetime.now().timestamp())
my_file = open("/scratch/shared/mrozemul/Fiji.app/extract_hyphae.sh", "w")
my_file.write("#!/bin/bash \n#Set job requirements \n#SBATCH -N 1 \n#SBATCH -t 2:00:00\n#SBATCH -p normal\n")
my_file.write(f'#SBATCH -o "/home/mrozemul/Corentin/MscThesis/slurm/node_id_{plate}_{begin}_{end}_{ide}.out" \n')
my_file.write(f'python ~/Corentin/MscThesis/hyphae_extraction.py {plate} {begin} {end+1}&\n')
my_file.write('wait\n')
my_file.close()
# !srun -t 2:00:00 --pty /scratch/shared/mrozemul/Fiji.app/extract_hyphae.sh
!sbatch /scratch/shared/mrozemul/Fiji.app/extract_hyphae.sh

Submitted batch job 9448603


***Check Hyphae***

In [15]:
begin = 0
end = 20
directory = "/scratch/shared/mrozemul/Fiji.app/" 
listdir=os.listdir(directory) 
list_dir_interest=[name for name in listdir if name.split('_')[-1]==f'Plate{0 if plate<10 else ""}{plate}']
ss=[name.split('_')[0] for name in list_dir_interest]
ff=[name.split('_')[1] for name in list_dir_interest]
dates_datetime=[datetime(year=int(ss[i][:4]),month=int(ss[i][4:6]),day=int(ss[i][6:8]),hour=int(ff[i][0:2]),minute=int(ff[i][2:4])) for i in range(len(list_dir_interest))]
dates_datetime.sort()
dates_datetime_chosen=dates_datetime[begin:end]
dates = [f'{0 if date.month<10 else ""}{date.month}{0 if date.day<10 else ""}{date.day}_{0 if date.hour<10 else ""}{date.hour}{0 if date.minute<10 else ""}{date.minute}' for date in dates_datetime_chosen]
exp = pickle.load( open(f'/scratch/shared/mrozemul/Fiji.app/Analysis_Plate{plate}_{dates[0]}_{dates[-1]}/experiment_{plate}.pick', "rb" ) )

In [16]:
len(exp.nodes)

9249

In [51]:
hyph = choice(exp.hyphaes)
hyph.ts

[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [52]:
hyph.end.ts()

[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [53]:
plt.close('all')
hyph.end.show_source_image(hyph.ts[-1],hyph.ts[-1])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [56]:
plt.close('all')
exp.plot([0,hyph.ts[-2],hyph.ts[-1]],[[hyph.root.label,hyph.end.label]]*3)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …